In [34]:
import nltk
import speech_recognition as sr

r = sr.Recognizer()
with sr.AudioFile("Full_Call.wav") as source:
    audio = r.record(source)
text = r.recognize_google(audio)
print(text)

hello thank you for calling Amazon customer support my name is Alex how can I assist you today I need to return a product I purchase last week sure I would be happy to help with that could you please provide your order number yes the order number is 56789 thank you just to confirm is this for the wireless headphones you purchase on January 15th yes that's correct I will process the return for you before we proceed I want to inform you of Return policy returns must be initiated within 1330 days of purchase and the item must be in its original packaging show your security please do not share any sensitive information like your credit card details account password during this school my credit card number with your call you yesterday 1234567 8987 6543200 I didn't know that I have initiated the return process you will receive a confirmation email within 24 hours and return label with will be sent to you registered email is there anything else I can help you with today no that's all thank yo

In [38]:
# Lists of phrases
greetings = [
    "hello", "hi", "good morning", "good afternoon", "good evening", 
    "greetings", "hey", "how can I help you?", "how may I assist you today?", 
    "welcome to", "thank you for calling", "this is", "thank you for reaching out.", 
    "welcome to our customer support"
]

closing_statements = [
    "thank you for calling", "have a great day", "goodbye", "take care", 
    "have a nice day", "stay safe", "thank you for choosing", 
    "if you have further questions, feel free to reach out", 
    "we’re here if you need any more help", "it was my pleasure assisting you", 
    "let us know if there’s anything else we can do for you", 
    "thank you" ,"goodbye"
]

disclaimers = [ 
    'please do not share any sensitive information',
    "for your security", "avoid disclosing personal information" "avoid disclosing personal information during this call", 
    "your privacy is important to us. do not share confidential details", 
    "returns must be initiated within", "by proceeding with this call, you agree to our terms and conditions", 
    "all calls may be monitored or recorded for quality and training purposes", 
    "we value your privacy and ensure all data is handled securely", 
    "please review our privacy policy on"
]

# Function to detect phrases in text
def detect_phrases(text, phrase_list):
    detected_phrases = []
    for phrase in phrase_list:
        if phrase.lower() in text.lower():
            detected_phrases.append(phrase)
    return detected_phrases


# Detecting phrases
detected_greetings = detect_phrases(text, greetings)
detected_closing_statements = detect_phrases(text, closing_statements)
detected_disclaimers = detect_phrases(text, disclaimers)

# Output results
print("Detected Greetings:", detected_greetings)
print("Detected Closing Statements:", detected_closing_statements)
print("Detected Disclaimers:", detected_disclaimers)


Detected Greetings: ['hello', 'hi', 'thank you for calling']
Detected Closing Statements: ['thank you for calling', 'have a great day', 'thank you for choosing', 'thank you']
Detected Disclaimers: ['please do not share any sensitive information', 'returns must be initiated within']


In [ ]:
from pyannote.audio import Pipeline
import speech_recognition as sr

# Initialize the pyannote pipeline for speaker diarization
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", use_auth_token="<HUGGING_FACE_TOKEN>")

# Input audio file
audio_file = "Full_Call.wav"

# Perform speaker diarization
diarization = pipeline(audio_file)

# Prepare the speech recognizer
recognizer = sr.Recognizer()

# Map diarized segments to speakers
speaker_segments = []

with sr.AudioFile(audio_file) as source:
    audio = recognizer.record(source)
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        # Extract audio segment for each speaker
        segment_audio = audio.get_segment(turn.start, turn.end)
        # Transcribe the segment
        try:
            text = recognizer.recognize_google(segment_audio)
        except sr.UnknownValueError:
            text = "[Unintelligible]"
        except sr.RequestError as e:
            text = f"[Error: {e}]"

        # Store speaker and their text
        speaker_segments.append({"speaker": speaker, "start": turn.start, "end": turn.end, "text": text})

# Display the results
for segment in speaker_segments:
    print(f"{segment['speaker']} [{segment['start']:.2f}s - {segment['end']:.2f}s]: {segment['text']}")
